# Mobile Customer Churn

In this Portfolio task you will work with some (fake but realistic) data on Mobile Customer Churn.  Churn is where
a customer leaves the mobile provider.   The goal is to build a simple predictive model to predict churn from available features. 

The data was generated (by Hume Winzar at Macquarie) based on a real dataset provided by Optus.  The data is simulated but the column headings are the same. (Note that I'm not sure if all of the real relationships in this data are preserved so you need to be cautious in interpreting the results of your analysis here).  

The data is provided in file `MobileCustomerChurn.csv` and column headings are defined in a file `MobileChurnDataDictionary.csv` (store these in the `files` folder in your project).

Your high level goal in this notebook is to try to build and evaluate a __predictive model for churn__ - predict the value of the CHURN_IND field in the data from some of the other fields.  Note that the three `RECON` fields should not be used as they indicate whether the customer reconnected after having churned. 

__Note:__ you are not being evaluated on the _accuracy_ of the model but on the _process_ that you use to generate it.  You can use a simple model such as Logistic Regression for this task or try one of the more advanced methods covered in recent weeks.  Explore the data, build a model using a selection of features and then do some work on finding out which features provide the most accurate results.  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [31]:
churn = pd.read_csv("files/MobileCustomerChurn.csv", na_values=["NA", "#VALUE!"], index_col='INDEX')
churn = churn.iloc[:, :-3] #remove last 3 RECON columns
churn.head()

,CUST_ID,ACCOUNT_TENURE,ACCT_CNT_SERVICES,AGE,CFU,SERVICE_TENURE,PLAN_ACCESS_FEE,BYO_PLAN_STATUS,PLAN_TENURE,MONTHS_OF_CONTRACT_REMAINING,LAST_FX_CONTRACT_DURATION,CONTRACT_STATUS,PREV_CONTRACT_DURATION,HANDSET_USED_BRAND,CHURN_IND,MONTHLY_SPEND,COUNTRY_METRO_REGION,STATE
INDEX,,,,,,,,,,,,,,,,,,
1,1,46,1,30.0,CONSUMER,46,54.54,NON BYO,15,0,24,OFF-CONTRACT,24,SAMSUNG,1,61.40,COUNTRY,WA
2,2,60,3,55.0,CONSUMER,59,54.54,NON BYO,5,0,24,OFF-CONTRACT,24,APPLE,1,54.54,METRO,NSW
3,5,65,1,29.0,CONSUMER,65,40.90,BYO,15,0,12,OFF-CONTRACT,12,APPLE,1,2.50,COUNTRY,WA
4,6,31,1,51.0,CONSUMER,31,31.81,NON BYO,31,0,24,OFF-CONTRACT,24,APPLE,1,6.48,COUNTRY,VIC
5,8,95,1,31.0,CONSUMER,95,54.54,NON BYO,0,0,24,OFF-CONTRACT,24,APPLE,1,100.22,METRO,NSW


In [14]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46206 entries, 1 to 46206
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   CUST_ID                       46206 non-null  int64  
 1   ACCOUNT_TENURE                46206 non-null  int64  
 2   ACCT_CNT_SERVICES             46206 non-null  int64  
 3   AGE                           46130 non-null  float64
 4   CFU                           46206 non-null  object 
 5   SERVICE_TENURE                46206 non-null  int64  
 6   PLAN_ACCESS_FEE               46206 non-null  float64
 7   BYO_PLAN_STATUS               46206 non-null  object 
 8   PLAN_TENURE                   46206 non-null  int64  
 9   MONTHS_OF_CONTRACT_REMAINING  46206 non-null  int64  
 10  LAST_FX_CONTRACT_DURATION     46206 non-null  int64  
 11  CONTRACT_STATUS               46206 non-null  object 
 12  PREV_CONTRACT_DURATION        46206 non-null  int64  
 13  H

In [13]:
reference = pd.read_csv("files/MobileChurnDataDictionary.csv")
pd.set_option('display.max_colwidth', None)
reference

,column_Number,Variable_name,Description
0,1,CUST_ID,Unique customer identifier
1,2,ACCOUNT_TENURE,Account tenure in months based on last day of reporting month
2,3,ACCT_CNT_SERVICES,Total number of services under an account
3,4,AGE,Age of the account holder (derived on dob)
4,5,CFU,Customer facing unit derived based on owning cost centre
5,6,SERVICE_TENURE,Service tenure in months
6,7,PLAN_ACCESS_FEE,Monthly access fee payable for the plan excluding gst
7,8,BYO_PLAN_STATUS,Whether the customer is on a BYO (bring your own) plan
8,9,PLAN_TENURE,Plan tenure in months
9,10,MONTHS_OF_CONTRACT_REMAINING,Contract tenure remaining based on months of contract completed


In [29]:
for col in churn:
    print(col, churn[col].unique())

CUST_ID [    1     2     5 ... 79496 79497 79500]
ACCOUNT_TENURE [ 46  60  65  31  95  62  90  12  13   9  47  24  49  75  33  10  25   4
  34  67  92  76  45  73  85  32  16   3  58  96  91  78  50  57  93  83
  30  94   8   7  23  89  80  27   0  97  63   6  69  64  87  54   1  51
  11  19  41  22  79  43  14   5  21   2  15  20  66  38  28  86  53  40
  71  84  42  55  48  81  36  26  39  88  44  61  17  59  82  99  29  98
  18  74  37  35  52  70  56  68  77  72 103 114 105 111 100 108 102 109
 107 110 104 116 101 119 112 115 106 118 113 120 117]
ACCT_CNT_SERVICES [1 3 2 4]
AGE [ 30.  55.  29.  51.  31.  25.  35.  66.  21.  41.  59.  22.  53.  47.
  50.  20.  48.  49.  26.  34.  68.  27.  56.  43.  44.  18.  60.  61.
  58.  23.  65.  33.  24.  45.  39.  64.  40.  19.  42.  52.  57.  54.
  46.  71.  32.  88.  79.  36.  72.  28.  62.  85.  70.  nan  69.  38.
  37.  77.  75.  63.  76.  73.  86.  16.  67.  89.  84. 116.  78.  74.
  91.  80.  82.  83.   5.  95.  87.  81. 115.  93.  92. 

In [24]:
churn.isna().sum()#any(axis=1)

CUST_ID                          0
ACCOUNT_TENURE                   0
ACCT_CNT_SERVICES                0
AGE                             76
CFU                              0
SERVICE_TENURE                   0
PLAN_ACCESS_FEE                  0
BYO_PLAN_STATUS                  0
PLAN_TENURE                      0
MONTHS_OF_CONTRACT_REMAINING     0
LAST_FX_CONTRACT_DURATION        0
CONTRACT_STATUS                  0
PREV_CONTRACT_DURATION           0
HANDSET_USED_BRAND               0
CHURN_IND                        0
MONTHLY_SPEND                    0
COUNTRY_METRO_REGION             1
STATE                            1
dtype: int64

In [28]:
churn.loc[churn.AGE.isna()]

,CUST_ID,ACCOUNT_TENURE,ACCT_CNT_SERVICES,AGE,CFU,SERVICE_TENURE,PLAN_ACCESS_FEE,BYO_PLAN_STATUS,PLAN_TENURE,MONTHS_OF_CONTRACT_REMAINING,LAST_FX_CONTRACT_DURATION,CONTRACT_STATUS,PREV_CONTRACT_DURATION,HANDSET_USED_BRAND,CHURN_IND,MONTHLY_SPEND,COUNTRY_METRO_REGION,STATE
INDEX,,,,,,,,,,,,,,,,,,
186,390,94,2,NaN,SMALL BUSINESS,94,45.44,NON BYO,27,0,24,OFF-CONTRACT,24,HUAWEI,1,51.79,COUNTRY,NSW
213,441,39,4,NaN,SMALL BUSINESS,1,36.36,NON BYO,1,0,0,NO-CONTRACT,0,SAMSUNG,1,48.37,METRO,NSW
2047,4454,4,2,NaN,SMALL BUSINESS,4,72.72,NON BYO,4,20,24,ON-CONTRACT,0,SAMSUNG,1,72.72,METRO,ACT
2560,5602,8,4,NaN,SMALL BUSINESS,8,27.27,BYO,5,0,0,NO-CONTRACT,0,APPLE,1,37.66,METRO,VIC
4129,9178,9,3,NaN,SMALL BUSINESS,9,72.72,NON BYO,9,15,24,ON-CONTRACT,0,APPLE,1,72.72,METRO,NSW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41919,72536,7,3,NaN,SMALL BUSINESS,5,27.27,BYO,5,0,0,NO-CONTRACT,0,SAMSUNG,1,72.56,METRO,VIC
42252,73087,94,2,NaN,SMALL BUSINESS,16,54.54,NON BYO,0,24,24,ON-CONTRACT,0,APPLE,0,91.35,METRO,VIC
44004,75927,0,1,NaN,SMALL BUSINESS,0,45.45,NON BYO,0,0,0,NO-CONTRACT,0,APPLE,0,45.45,METRO,WA


# Clean the data

In [39]:
with pd.option_context('display.max_columns', None): #force to display all data
    print(pd.get_dummies(churn))

       CUST_ID  ACCOUNT_TENURE  ACCT_CNT_SERVICES   AGE  SERVICE_TENURE  \
INDEX                                                                     
1            1              46                  1  30.0              46   
2            2              60                  3  55.0              59   
3            5              65                  1  29.0              65   
4            6              31                  1  51.0              31   
5            8              95                  1  31.0              95   
...        ...             ...                ...   ...             ...   
46202    79494              10                  1  61.0               9   
46203    79495               2                  1  53.0               2   
46204    79496              47                  1  22.0              47   
46205    79497              94                  1  40.0             182   
46206    79500               0                  1  24.0               0   

       PLAN_ACCESS_FEE  